In [ ]:
#Kalpa

In [ ]:
#Arjun

## Polina's Model

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import export_text, DecisionTreeClassifier, plot_tree

In [6]:
import pickle 

In [10]:
# Import Gensim.
import gensim
from gensim.models.word2vec import Word2Vec

In [188]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/polinaminkovski/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
from sklearn.decomposition import PCA

In [7]:
with open('training.pkl', 'rb') as file: 
      
    df_train = pickle.load(file) 

In [8]:
with open('testing.pkl', 'rb') as file: 
      
    df_test = pickle.load(file) 

In [23]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Cleaning

In [24]:
df_train["text_lower"] = df_train['text'].str.lower()

In [118]:
df_train['tokenized_lower_no_char'] = df_train['text_lower'].str.replace('[^a-zA-Z0-9\s]+',"", regex=True)

In [121]:
df_train['tokenized_text'] = df_train.apply(lambda row: nltk.word_tokenize(row['tokenized_lower_no_char']), axis=1)

In [193]:
tweets = []
labels = []

for index, row in df_train.iterrows():
  labels.append(row['target'])

  token_list = [x for x in row['tokenized_text'] if x not in stopwords.words('english')]

  tweets.append(" ".join(token_list))


In [168]:
df_train['split_text'] = [sentence.split() for sentence in df_train["tokenized_lower_no_char"]]

In [209]:
X_w2v = df_train['split_text']
with open('X_w2v.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(X_w2v, file) 

In [ ]:
with open('training.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(df_train, file) 

In [4]:
#model = gensim.models.KeyedVectors.load_word2vec_format('lexvec.commoncrawl.300d.W.pos.vectors.gz')

In [196]:
X = tweets
y = labels

In [206]:
with open('X.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(X, file) 

In [207]:
with open('y.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(y, file) 

### More cleaning

In [210]:
df_train['lower_some_char'] = df_train['text_lower'].str.replace('[^a-zA-Z0-9#@\s]+',"", regex=True)

In [211]:
tweets_c = []
labels_c = []

for index, row in df_train.iterrows():
  labels_c.append(row['target'])

  token_list = [x for x in row['lower_some_char'] if x not in stopwords.words('english')]

  tweets_c.append(" ".join(token_list))

In [212]:
X_c = tweets_c
y_c = labels_c

In [213]:
with open('X_c.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(X_c, file) 
    
with open('y_c.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(y_c, file) 

In [197]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

In [198]:
model1 = gensim.models.Word2Vec(X_train, min_count = 1,vector_size = 100, window = 5, sg=0) 

In [199]:
vocabulary = list(model1.wv.key_to_index)

In [200]:
def vectorize(text_):
    words = text_
    words_vecs = [model1.wv[word] for word in words if word in vocabulary]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [203]:
X_train_w2v = np.array([vectorize(text_) for text_ in X_train])
with open('X_train_w2v.pkl', 'wb') as file: 
      
    pickle.dump(X_train_w2v, file) 

In [204]:
X_val_w2v = np.array([vectorize(text_) for text_ in X_val])
with open('X_val_w2v.pkl', 'wb') as file: 
      
    pickle.dump(X_val_w2v, file) 